In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 28 14:37:03 2023

@author: aksha
"""

import requests
import pandas as pd
import os

# Define the FRED API endpoint and your API key
api_endpoint = 'https://api.stlouisfed.org/fred/series/observations'
api_key = '4014619263373f027c79b088ce24c4fd'  # Replace 'YOUR_API_KEY' with your actual FRED API key

# Function to fetch data from FRED API
def fetch_fred_data(series_id, start_date, end_date):
    """
    Fetches data from the FRED API for a given series ID, start date, and end date.
    
    Parameters:
    - series_id (str): The ID of the series to fetch data for.
    - start_date (str): The start date of the data range.
    - end_date (str): The end date of the data range.
    
    Returns:
    - df (pandas.DataFrame): The fetched data as a DataFrame, or None if the request failed.
    """
    api_endpoint = 'https://api.stlouisfed.org/fred/series/observations'
    params = {
        'series_id': series_id,
        'api_key': api_key,
        'file_type': 'json',
        'observation_start': start_date,
        'observation_end': end_date,
    }
    
    response = requests.get(api_endpoint, params=params)
    
    if response.status_code == 200:
        data = response.json()
        observations = data['observations']
        df = pd.DataFrame(observations)
        return df
    else:
        print(f"Failed to fetch data for {series_id}: {response.status_code}")
        return None

# Fetch data for GDP
# Specify the directory path
directory = 'C:/Users/aksha/Downloads/data_eng_fin/external data'

# Fetch data for GDP
gdp_data = fetch_fred_data('GDP', '2013-01-01', '2023-10-01')
gdp_data.to_csv(os.path.join(directory, 'gdp_data.csv'), index=False)

# Fetch data for Unemployment
unemployment_data = fetch_fred_data('UNRATE', '2013-01-01', '2023-10-01')
unemployment_data.to_csv(os.path.join(directory, 'unemployment_data.csv'), index=False)

# Fetch data for CPI (US total inflation)
cpi_data = fetch_fred_data('USACPALTT01CTGYM', '2013-01-01', '2023-10-01')
cpi_data.to_csv(os.path.join(directory, 'cpi_data.csv'), index=False)

# Fetch data for Core CPI
non_core_cpi_data = fetch_fred_data('CPGRLE01USM657N', '2013-01-01', '2023-10-01')
non_core_cpi_data.to_csv(os.path.join(directory, 'non_core_cpi_data.csv'), index=False)

# Ensure data is fetched successfully before proceeding
if cpi_data is not None and non_core_cpi_data is not None:
    # Convert 'value' column to numeric
    cpi_data['value'] = pd.to_numeric(cpi_data['value'])
    non_core_cpi_data['value'] = pd.to_numeric(non_core_cpi_data['value'])
    
    # Calculate core CPI by subtracting non-core CPI from overall CPI
    core_cpi_data = cpi_data.copy()
    core_cpi_data['value'] = cpi_data['value'] - non_core_cpi_data['value']
    
    # Save core CPI data to a CSV file
    core_cpi_data.to_csv(os.path.join(directory, 'core_cpi_data.csv'), index=False)
    print("Core CPI data calculated and saved.")
else:
    print("Data fetching failed. Check the API or internet connection.")


Core CPI data calculated and saved.


In [3]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 28 16:44:38 2023

@author: aksha
"""

import pandas as pd
from scipy.interpolate import interp1d
import numpy as np
import os

# Define the FRED API endpoint and your API key
api_endpoint = 'https://api.stlouisfed.org/fred/series/observations'
api_key = '4014619263373f027c79b088ce24c4fd'  # Replace 'YOUR_API_KEY' with your actual FRED API key

# Function to fetch data from FRED API
def fetch_fred_data(series_id, start_date, end_date):
    """
    Fetches data from the FRED API for a given series ID, start date, and end date.
    
    Parameters:
    - series_id (str): The ID of the series to fetch data for.
    - start_date (str): The start date of the data range.
    - end_date (str): The end date of the data range.
    
    Returns:
    - df (pandas.DataFrame): The fetched data as a DataFrame, or None if the request failed.
    """
    params = {
        'series_id': series_id,
        'api_key': api_key,
        'file_type': 'json',
        'observation_start': start_date,
        'observation_end': end_date,
    }
    
    response = requests.get(api_endpoint, params=params)
    
    if response.status_code == 200:
        data = response.json()
        observations = data['observations']
        df = pd.DataFrame(observations)
        return df
    else:
        print(f"Failed to fetch data for {series_id}: {response.status_code}")
        return None

# Function to generate business days only
def generate_business_days(start_date, end_date):
    return pd.date_range(start=start_date, end=end_date, freq='B')

# Define a function for spline interpolation on business days
def interpolate_to_business_days(df):
    # Create a spline interpolation function
    f = interp1d(df.index.astype(np.int64), df['value'], kind='cubic', axis=0)
    
    # Generate business days timestamps from the minimum to maximum date
    business_days_index = generate_business_days(df.index.min(), df.index.max())
    
    # Interpolate values at business days intervals
    interpolated_values = f(business_days_index.astype(np.int64))
    
    # Create a new DataFrame with the business days index and interpolated values
    interpolated_df = pd.DataFrame({'value': interpolated_values}, index=business_days_index)
    
    return interpolated_df


Data upscaled to business days only using spline interpolation and saved to CSV files.
Data upscaled to business days only using spline interpolation and saved to CSV files.
